In [39]:
import pandas as pd
import thinkplot
import numpy as np
import re
from tqdm import tqdm
# from reporter import Reporter
import random
import time

In [26]:
class Reporter(object):
    def __init__(self, name):
        self.name = name
        self.cnt = 0
        self.cliques = []

    def inc_count(self):
        self.cnt += 1

    def record(self, clique):
        self.cliques.append(clique)

    def sort_cliques(self):
        self.cliques.sort(key=len, reverse=True)

    def print_max(self, n):
        # print the n largest cliques
        self.sort_cliques()
        print(self.name)
        print('%d recursive calls' % self.cnt)
        for i in range(n):
            clique = self.cliques[i]
            print('%d: %s' % (i, clique))
        print()

    def print_report(self):
        print(self.name)
        print('%d recursive calls' % self.cnt)
        for i, clique in enumerate(self.cliques):
            print('%d: %s' % (i, clique))
        print()


Read in `Amazon0302.txt` as an adjacency matrix. First read in the text file as a Dataframe.

Info about Amazon data:
- Directed graph (each unordered pair of nodes is saved once): Amazon0302.txt 
- Amazon product co-purchaisng network from March 02 2003
- Nodes: 262111 Edges: 1234877

In [2]:
df_transition = pd.read_csv("20191116_Amazon0302.txt",delimiter="\t", skiprows=[0,1,2])
df_transition.head()

,# FromNodeId,ToNodeId
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [3]:
df_transition.columns = ['start', 'end']
df_transition.head()

,start,end
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


Build adjacency matrix in the form of a dict with vertexes as keys and lists of their neighbors as values

In [4]:
graph = {}

for index, row in df_transition.iterrows():
    if index%100000==0:
        print(index)
    start = row['start']
    end = row['end']
    if start in graph and end not in graph[start]:
        graph[start].append(end)
    elif start not in graph:
        graph[start] = [end]
        
    if end in graph and start not in graph[end]:
        graph[end].append(start)
    elif end not in graph:
        graph[end] = [start]

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000


Implement naive Bron–Kerbosch algorithm

In [27]:
def bronKerbosch1(clique, candidates, excluded, reporter):
    '''Naive Bron–Kerbosch algorithm'''
    reporter.inc_count()
    if not candidates and not excluded:
        if len(clique) >= 3:
            reporter.record(clique)
        return
    
    for v in list(candidates):
        new_candidates = candidates.intersection(graph[v])
        new_excluded = candidates.intersection(graph[v])
        bronKerbosch1(clique+[v], new_candidates, new_excluded, reporter)
        candidates.remove(v)
        excluded.add(v)

In [40]:
start = time.time()
report1 = Reporter('## %s' % bronKerbosch1.__doc__)
bronKerbosch1([], set(graph.keys()), set(), report1)
end = time.time()
print('Naive method:', end - start)

Naive method: 12.213986873626709


In [41]:
report1.print_max(10)

## Naive Bron–Kerbosch algorithm
2258288 recursive calls
0: [330, 718, 720, 377, 721, 722, 723]
1: [2447, 4673, 4674, 4675, 4676, 4677, 5119]
2: [2696, 6017, 6018, 6288, 6019, 6020, 6021]
3: [9093, 27018, 23599, 40440, 40437, 40438, 40439]
4: [13690, 22931, 20325, 22264, 22265, 39388, 22263]
5: [29489, 51265, 51266, 119227, 51267, 108853, 32870]
6: [32210, 42074, 43258, 75482, 42075, 43259, 42076]
7: [39031, 51014, 51015, 39032, 46680, 46996, 46679]
8: [41605, 113090, 59366, 59368, 59369, 59370, 59367]
9: [65139, 83105, 83107, 94846, 94252, 94253, 94254]



Implement Bron–Kerbosch algorithm with a pivot. Randomly pick a vertex u from `candidates` or `excluded`. The maximal clique must include either u or one of its non-neighbors.

In [30]:
def bronKerbosch2(clique, candidates, excluded, reporter):
    '''Bron–Kerbosch algorithm with pivot'''
    reporter.inc_count()
    if not candidates and not excluded:
        if len(clique) >= 3:
            reporter.record(clique)
        return
    u = pick_pivot(candidates) or pick_pivot(excluded)
    # only consider u or its non neighbors
    for v in list(candidates.difference(graph[u])):
        new_candidates = candidates.intersection(graph[v])
        new_excluded = candidates.intersection(graph[v])
        bronKerbosch2(clique+[v], new_candidates, new_excluded, reporter)
        candidates.remove(v)
        excluded.add(v)
        
def pick_pivot(nodes):
    if nodes:
        return random.sample(nodes, 1)[0]

In [42]:
start = time.time()
report2 = Reporter('## %s' % bronKerbosch2.__doc__)
bronKerbosch2([], set(graph.keys()), set(), report2)
end = time.time()
print('Pivot method:', end - start)

Pivot method: 16.47268581390381


In [43]:
report2.print_max(10)

## Bron–Kerbosch algorithm with pivot
1627022 recursive calls
0: [330, 718, 721, 722, 720, 377, 723]
1: [2447, 5119, 4674, 4675, 4677, 4676, 4673]
2: [2696, 6018, 6020, 6017, 6288, 6019, 6021]
3: [9093, 40439, 40438, 40440, 27018, 40437, 23599]
4: [13690, 22931, 22264, 39388, 20325, 22263, 22265]
5: [29489, 119227, 108853, 51267, 51265, 32870, 51266]
6: [32210, 42076, 75482, 43259, 42075, 43258, 42074]
7: [39031, 51014, 46680, 46679, 51015, 39032, 46996]
8: [41605, 59367, 59366, 59370, 59369, 113090, 59368]
9: [65139, 94254, 83105, 94252, 94253, 94846, 83107]



Implement Bron–Kerbosch algorithm with a pivot and degeneracy ordering. Degeneracy ordering is the ordering of vertices such that each vertex has d or fewer neighbors that come later in the ordering. Select vertex of minimum degree among remaining vertices.

In [35]:
def bronKerbosch3(clique, candidates, excluded, reporter):
    '''Bron–Kerbosch algorithm with pivot and degeneracy ordering'''
    reporter.inc_count()
    if not candidates and not excluded:
        if len(clique) >= 3:
            reporter.record(clique)
        return
    for v in list(degeneracy_order(candidates)):
        new_candidates = candidates.intersection(graph[v])
        new_excluded = candidates.intersection(graph[v])
        bronKerbosch2(clique.append(v), new_candidates, new_excluded, reporter)
        candidates.remove(v)
        excluded.add(v)
        
def degeneracy_order(nodes):
    deg = {}
    for node in graph:
        deg[node] = len(graph[node])
    
    while deg:
        # find min degree
        i, d = min(deg.items(), key=lambda pair:pair[1])
        yield i
        del deg[i]
        for v in graph[i]:
            if v in deg:
                deg[v] -= 1

In [36]:
report3 = Reporter('## %s' % bronKerbosch3.__doc__)
bronKerbosch3([], set(graph.keys()), set(), report3)

KeyboardInterrupt: 

In [ ]:
report3.print_max(10)